In [2]:
import numpy as np
import pandas as pd
import benchmarking as bm

import warnings
warnings.filterwarnings('ignore')

## Individual Model Time Benchmarks Benchmarks

This notebook demonstratces how the library can be used to benchmark the time performance of various models. We first show the timing method for a single model on a single run as a proof of concept and sainty check

In [3]:
inference_times = []
train_times = []

smallest_qubits_circuit_size = N0 = 10
largest_qubit_circuit_size = N = 16


for num_qubits in range(N0,N+2,2):
    
    model = ### INSERT MODEL AS FUNCTION OF QUBITS
    
    X, y = bm.make_data.gen_sets(dims=num_qubits//2,npoints=1)
    
    inference_time = bm.timing.inference(model, X, y)
    train_time = bm.timing.train(model, X, y)
    
    inference_times.append(round(inference_time,4))
    train_times.append(round(train_time,4))

qubit_numbers = [f"{num_qubits} qubits" for num_qubits in range(N0,N+2,2)] 
    
print("MODEL NAME:")
print("Inference Times:", dict(zip(qubit_numbers,inference_times)))
print("Training Times:", dict(zip(qubit_numbers,train_times)))

MODEL NAME:
Inference Times: {'10 qubits': 0.0045, '12 qubits': 0.0042, '14 qubits': 0.0102, '16 qubits': 0.0484}
Training Times: {'10 qubits': 0.0053, '12 qubits': 0.0049, '14 qubits': 0.0093, '16 qubits': 0.0478}


## Many Model Time Benchmarks Benchmarks

Here we loop over multiple models with many runs and take averages and standard deviations of the runtime. We pass this data through chauvenet's criteria to address outliers. Finally we present both the training and inference times in a pandas DataFrame table.

In [4]:
model_list = [
    ### Insert Models Here
]

model_names =[
    'Model1',
    'Model2',
    'Model3',
    'Model4']

models = dict(zip(model_names, model_list))

smallest_qubits_circuit_size = N0 = 2
largest_qubit_circuit_size = N = 20

runs = 10

#set thread count
set_omp_thread_number(48)

#all runs
inference_times = np.zeros((len(models), N//2, runs))
train_times = np.zeros((len(models), N//2, runs))

#mean &std
inference_time_stats = np.zeros((len(models), N//2, 2))
train_time_stats = np.zeros((len(models), N//2, 2))

In [5]:
for j, num_qubits in enumerate(range(N0, N + 2,2)):
    X, y = bm.gen_sets(dims=num_qubits//2,)
    for i, model in enumerate(models.values()):
        model = model(num_qubits)
        for r in range(runs):
            inference_times[i,j,r] = bm.timing.inference(model, X, y)
            train_times[i,j,r] = bm.timing.train(model, X, y)
        #apply chauvenet criteria
        inference_time_stats[i,j,:] = bm.chauvenet_stats(inference_times[i,j,:]) 
        train_time_stats[i,j,:] = bm.chauvenet_stats(train_times[i,j,:])
    print(f'Testing {len(models)} models with {num_qubits} Qubits')
            
inference_df = bm.make_stats_table(inference_time_stats, models, N)
train_df = bm.make_stats_table(train_time_stats, models, N)
    

Testing 4 models with 2 Qubits
Testing 4 models with 4 Qubits
Testing 4 models with 6 Qubits
Testing 4 models with 8 Qubits
Testing 4 models with 10 Qubits
Testing 4 models with 12 Qubits
Testing 4 models with 14 Qubits
Testing 4 models with 16 Qubits
Testing 4 models with 18 Qubits
Testing 4 models with 20 Qubits


In [6]:
inference_df

,2 Qubits Mean,2 Qubits STD,4 Qubits Mean,4 Qubits STD,6 Qubits Mean,6 Qubits STD,8 Qubits Mean,8 Qubits STD,10 Qubits Mean,10 Qubits STD,12 Qubits Mean,12 Qubits STD,14 Qubits Mean,14 Qubits STD,16 Qubits Mean,16 Qubits STD,18 Qubits Mean,18 Qubits STD,20 Qubits Mean,20 Qubits STD
Model1,0.063640,0.000545,0.001443,0.000005,0.001840,0.000026,0.002239,0.000014,0.002727,0.000010,0.003695,0.000009,0.008277,0.000035,0.038169,0.000410,0.188272,0.001294,0.745843,0.007015
Model2,0.062794,0.000637,0.001489,0.000007,0.001885,0.000016,0.002303,0.000036,0.002782,0.000020,0.003733,0.000009,0.008376,0.000076,0.037979,0.000538,0.188632,0.001818,0.749993,0.004524
Model3,0.001195,0.000100,0.002127,0.000058,0.002413,0.000023,0.003304,0.000037,0.005201,0.000732,0.007673,0.000156,0.018504,0.000260,0.302306,0.211492,1.225515,0.237733,2.360398,0.184344
Model4,0.001201,0.000009,0.002199,0.000011,0.002478,0.000005,0.003391,0.000019,0.004668,0.000033,0.007563,0.000034,0.018275,0.000052,0.459588,0.111549,0.990127,0.317910,2.242169,0.259626


In [7]:
train_df

,2 Qubits Mean,2 Qubits STD,4 Qubits Mean,4 Qubits STD,6 Qubits Mean,6 Qubits STD,8 Qubits Mean,8 Qubits STD,10 Qubits Mean,10 Qubits STD,12 Qubits Mean,12 Qubits STD,14 Qubits Mean,14 Qubits STD,16 Qubits Mean,16 Qubits STD,18 Qubits Mean,18 Qubits STD,20 Qubits Mean,20 Qubits STD
Model1,0.065915,0.000217,0.002240,0.000025,0.002663,0.000013,0.003077,0.000009,0.003612,0.000035,0.004616,0.000044,0.009174,0.000069,0.039393,0.000754,0.190617,0.002753,0.743760,0.003304
Model2,0.067023,0.002130,0.002515,0.000040,0.002935,0.000060,0.003373,0.000047,0.003877,0.000055,0.004868,0.000034,0.009583,0.000134,0.039605,0.000565,0.190477,0.001790,0.746586,0.006421
Model3,0.001556,0.000066,0.002630,0.000079,0.002784,0.000007,0.003700,0.000024,0.004958,0.000056,0.007928,0.000039,0.018855,0.000146,0.260608,0.182814,1.008676,0.258785,2.115883,0.237554
Model4,0.001874,0.000002,0.003086,0.000102,0.003183,0.000010,0.004111,0.000024,0.005370,0.000019,0.008341,0.000037,0.019068,0.000030,0.367581,0.244299,1.168426,0.253939,2.291318,0.147852
